In [84]:
import os
import pyNetLogo
import matplotlib.pyplot as plt
import pandas as pd
#DED Note: Need to determine a distribution for mid-year population distribution 

import os
netlogo_home = "/netlogo-5.3.1-64"
netlogo_version = "5"
#os.chdir("C:\\Users\\DDawso01\\OneDrive - Environmental Protection Agency (EPA)\\Profile\\Documents\\Bee Project\\Beehave_PEEM_Nectar")
os.chdir("/home/ddawso01/Documents/BeeProject/BeeHave_PEEM_Nectar_DED")
#netlogo_home = "/netlogo-5.3.1-64"
#netlogo_version = "5"
Suff="DED_123119"
params=3

probparams = {
'num_vars': 3,
'names': [
'NectarPesticideInput',
'NectarSugarConcentrationInput',
'NectarVolumeInput'
],
'bounds': [
[0, 1000],
[0, 2.0],
[0, 10000.0],
]
}

from SALib.sample import saltelli
from SALib.analyze import sobol
n=1000
param_values=saltelli.sample(probparams, n, calc_second_order=True)


In [85]:
#Specification of Constants and/or variable things not of interest, including weather, food, start day, random-seed and runlength
import numpy as np
#Note: files are looked for in the directory of the jupyter notebook instead of the directory specified below via chdir. Thats peculiar; hopefully it won't be a problem when it is converted to a .py file 
#For some reason, having trouble specifying text files directly, will have to deal with later 
#weathervec=np.repeat("\"Weather_WI_08_2016_input.txt\"",  param_values.shape[0])
#foodinputvec=np.repeat("\"EnvFile_1flowerpatch_fromPython.txt\"",  param_values.shape[0])
startday=np.repeat(0, param_values.shape[0])
runlength=np.repeat(365, param_values.shape[0])
randomseed = np.random.uniform(0,10000,param_values.shape[0])
paramindex = list(range(1,(param_values.shape[0]+1))) #Have to put list around range to produce the range of numbers; don't need list inside a loop becuase its an iterator
#param_values=np.column_stack((param_values, foodinputvec, startday,runlength))
#param_values=np.column_stack((param_values, weathervec, foodinputvec, startday,runlength))
param_values=np.column_stack((param_values, startday,runlength, randomseed, paramindex))

#Note: np.vstack can be used similarly to add a row

In [86]:
#Creating Final Problem Set with Input File constants
#problem = {
#'num_vars': 7,
#'names': [
#'NectarPesticideInput',
#'NectarSugarConcentrationInput',
#'NectarVolumeInput', 'Input_File', 'WeatherFile', 'StartDay', 'X_Days'
#]}
problem = {
'num_vars': 7,
'names': [
'NectarPesticideInput',
'NectarSugarConcentrationInput',
'NectarVolumeInput','StartDay', 'X_Days', 'random-seed', 'paramindex'
]}





In [87]:
pv=pd.DataFrame(param_values)
pv.columns=['NectarPesticideInput','NectarSugarConcentrationInput','NectarVolumeInput', 
            'StartDay', 'X_Days', 'random_seed', 'paramindex']

SenParamLabel="SobalSA_paramsets" + str(params) + "_params_" + str(n) + "reps_" + str((2^3) * n) + "pmts_" +  Suff + ".csv"

#Check distributions
#plt.subplot(3,1,1)
#pv.iloc[:,1].plot(kind="hist", title="NectarPesticideInput")
#plt.subplot(3,1,2)
#pv.iloc[:,2].plot(kind="hist", title="NectarSugarConcentrationInput")
#plt.subplot(3,1,3)
#pv.iloc[:,3].plot(kind="hist", title="NectarVolumeInput")
pv.to_csv(SenParamLabel)


In [88]:
pv.shape

(8000, 7)

In [89]:
def simulation(experiment):

    #Set the input parameters
    for i, name in enumerate(problem['names']):
        if name == 'random-seed':
            #The NetLogo random seed requires a different syntax
            netlogo.command('random-seed {}'.format(experiment[i]))
        
        elif name == 'paramindex':
            pass
        
        else:
            #Otherwise, assume the input parameters are global variables
            netlogo.command('set {0} {1}'.format(name, experiment[i]))

    netlogo.command('setup')
    
    #Run for the specified number of ticks and return the values of the variables specified at each timestep
    #Note that this outputs a dataframe, which is transferred into a list per output with parallel processing
    counts = netlogo.repeat_report(['TotalEggs', 'TotalLarvae', 'TotalPupae', 'TotalIHBees', 
                                    'TotalForagers', 'AFF', 'NewWorkerEggs', 'HoneyEnergyStore', 
                                    'HoneyStorePesticideConc'], 365)


    return counts
    

In [90]:
import ipyparallel

client = ipyparallel.Client()

In [91]:
direct_view = client[:]

In [92]:
import os
cwd=os.getcwd()
os.chdir(cwd)
#Push the current working directory of the notebook to a "cwd" variable on the engines that can be accessed later
direct_view.push(dict(cwd=os.getcwd()))
direct_view.push(dict(problem=problem))

<AsyncResult: _push>

In [93]:
%%px
import os 
import pyNetLogo
import pandas as pd
netlogo_home = "/home/ddawso01/Documents/BeeProject/netlogo-5.3.1-64"
netlogo_version = "5"
netlogo=pyNetLogo.NetLogoLink(gui=False, netlogo_home=netlogo_home, netlogo_version=netlogo_version)
netlogo.load_model("/home/ddawso01/Documents/BeeProject/BeeHave_PEEM_Nectar_DED/Beehave_PEEM_Nectar_DED121919.nlogo")



#Setup Netlogo
#import os
##os.chdir(cwd)
#import pyNetLogo
#import pandas as pd
#netlogo_home = "/home/ddawso01/Documents/BeeProject/netlogo-5.3.1-64"
#netlogo_version = "5"
#netlogo = pyNetLogo.NetLogoLink(gui=False, netlogo_home=netlogo_home, netlogo_version=netlogo_version)
#netlogo.load_model("/home/ddawso01/Documents/BeeProject/BEEHAVE-Model_2014-03-04/1_BEEHAVE-MODEL_Beehave2013.nlogo")




In [94]:
%%time
lview = client.load_balanced_view()
results = lview.map_sync(simulation, param_values)
#Note, it took 2 min 11s(131 sec) for 800 simulations on 30 cores; thats 0.163 seconds per sim; 
#This took 18 min 1 s for 8000 simulations 

CPU times: user 1min 18s, sys: 29.8 s, total: 1min 48s
Wall time: 18min 1s


In [95]:
#Loop over elements of the list and create a large dataframe

resultdf=results[0]
resultdf['paramindex'] = np.repeat(1, resultdf.shape[0])
for i in range(2, (len(results))):
    resultdf1=results[i]
    resultdf1['paramindex'] = np.repeat(i, resultdf1.shape[0])
    resultdf=resultdf.append(resultdf1) # can be used similarly to add a row


In [96]:
#SenResults="SobalSenResults_1year_3params_1000reps_8000_Parameterizations_" + Suff + ".csv"
SenResults="SobalSA_Sims_" + str(params) + "params_" + str(n) + "reps_" + str((2^3) * n) + "pmts_" +  Suff + ".csv"

resultdf.to_csv(SenResults)